# Dijkstra算法
一个有向图，问：从出发点到各节点的最短距离，返回表。(可以出现距离正无穷)

分析：
- 先准备一张初始距离表，A—A的距离为0，A—X的距离为Inf
- 选取初始表中最小的那个值(很明显应该选A-A),以这个A为初始点开始找寻到各节点的最短路径(由于是第一次更新, 就是A的所有邻居节点)
- 从上次更新得到的距离表中选取最短距离的节点，比如选择到了C(此时可以断言, 这个距离一定是A到C的最短距离了4，不可能有其它A到C的路径更短), 以此节点开始跳转一次，计算所有C的邻居, 如果 C-邻居的距离 + A-C的距离 < A-邻居的距离，则说明A到邻居的距离可以通过C的跳转来变得更小，由此更新表。
- 重复以上步骤直到最后一个节点被使用(我们使用usedSet来记录已经使用过的点)。
- 按步骤运行可以之后知道，其实usedSet储存的就是作为“跳板”的点。当所有的点都在usedSet中时, 我们就可以说从起点startNote(A)到其他点最短路径已经被距离表记录了(相当于我们使用了所有的点作为跳板去遍历得到A到其他点的最短距离)

In [13]:
# 通过字典来得到权有向图
graph = {
    "A":{"B":5, "C":1},              # 字典，和A相连的是B(5)，C(1)
    "B":{"A":5, "C":2, "D":1},       # 字典，和B相连的是A(5)，C(2),D(1)
    "C":{"A":1, "B":2, "D":4, "E":8},# 字典，和C相连的是A(1)，B(2),D(4),E(8)
    "D":{"B":1, "C":4, "E":3, "F":6},# 字典，和D相连的是B(1)，C(4),E(3),F(8)
    "E":{"C":8, "D":3},              # 字典，和E相连的是C(8)，D(3)
    "F":{"D":6}                      # 字典，和F相连的是D(6)
}

# 将返回当前updateMap中最小的点以及其基准距离
def getMinNote(updateMap, usedSet):
    '''
    usedSet: set()用于记录使用过的点
    updateMap: 到各节点的距离字典
    '''
    minimun = float("inf") 
    minNote = None 
    for i in updateMap :
       if (i not in usedSet) and (updateMap[i] < minimun):
           minNote = i 
           minimun = updateMap[i]
    return minNote, minimun

# Dijkstra算法
def Dijkstra(startNote, graph):
    updateMap = {startNote:0}    #放入初始点
    usedSet = set()  #记录使用过的点
    minNote, distance = getMinNote(updateMap, usedSet)

    while minNote != None:      # 当useSet中已经完成了所有节点，则不用再循环了
        edges = graph[minNote]  # 得到的是一个字典，表示minNote能到各个点的距离{X:int...}
        for note in edges:
            if note not in updateMap : # 如果更新字典中不存在这条边
                updateMap[note] = edges[note] + distance
            else :
                # if条件可以删掉
                if note not in usedSet: # startNote到useSet中的点的最短路径已经被updateMap储存了，不可能有更短的路径了，因此可以不用更新了
                    updateMap[note] = min(distance+edges[note], updateMap[note])

        usedSet.add(minNote)
        minNote, distance = getMinNote(updateMap, usedSet)

    return updateMap
    
Dijkstra("C", graph)


{'C': 0, 'A': 1, 'B': 2, 'D': 3, 'E': 6, 'F': 9}

## Dijkstra算法改进(采用小根堆/优先队列)
在上述算法中有一步骤是我们需要找到距离表中最短距离的那个节点, 为此我们每次都要去遍历查询距离表(也就是getMinNote所做的事情), 我们可不可以使用某种手段在每次循环时直接得到我们需要的那个节点，以及对应的长度呢？

答：采用小根堆来存储记录，这样就不用遍历整个需要更新的字典来找到最小的距离了。

分析:
- 小根堆的特性：每次插入一个元素，会自动调整堆的结构，保证堆中元素的最小值在堆的根节点。
- 在python中自带的heap可以创建一个元组堆, 每个元素为(value,key)，也就是把带数字的元组类型推入堆中，堆会根据value对元素进行排序，使得其仍然是堆。因此我们使用一个堆pqueue用来存放需要进行排序的节点(也就是还没有被使用过的节点)，
- 每次循环，从堆中可以弹出距离最小的那个节点

优化:
- 更通用的模板, 不需要记录已经遍历过的点
```python
import heapq

dis = [inf] * n       # 表示起始点距离其余点的最短路径
dis[start] = cost     # 初始点的代价
h = [(cost, start)]   # 用于存放还没被完全更新的点
while h:
    # 已经是起始点到x的最短路径了
    dis_x, x = heappop(h)
    # 如果当前点不是起始点到x的最短路径, 跳过(因为可能x已经在出过堆了, 这个是从其他路径放入到h中的)
    if dis_x > dis[x]:
        continue
    # 遍历x的所有邻居
    for y, w in graph[x]:
        new_dis = dis_x + w   # 起始点到y的最短路径
        if new_dis < dis[y]:  # 如果经过x中转可以使得起始点到y的最短路径更短, 更新起始点到y的最短路径, 并把y放入堆中
            dis[y] = new_dis
            heappush(h, (new_dis, y))
```

In [3]:
from heapq import * 
from collections import defaultdict
import math

def Dijkstra(startNode, graph):
    minDistance = defaultdict(lambda: math.inf)  # 默认距离其他点为正无穷
    minDistance[startNode] = 0   # 初始点为距离为0
    pqueue = []                  # 用来存放还没被完全更新的节点
    heappush(pqueue, (0, startNode)) 

    while pqueue:      # 当pqueue为空时意味着已经被更新完毕了，则不用再循环了
        distance, minNode = heappop(pqueue)
        if distance > minDistance[minNode]:  # minNode之前出过堆, 跳过
            continue

        # 遍历minNode相连的点, 这些点可能通过minNode的转移从而实现距离起始点更小的距离
        for node, wight in graph[minNode]:
            new_distance = distance + wight
            if new_distance < minDistance[node]: # 如果通过这个点中转后可以使得距离起始点更近则更新
                minDistance[node] = new_distance
                # 这个node由于被更新了, 因此其他点可能通过它作为中转链的点，因此需要重新入堆
                heappush(pqueue, (new_distance, node))
    return minDistance
    
graph = {
    "A": [("B",5), ("C", 1)],                   # 字典，和A相连的是B(5)，C(1)
    "B": [("A",5), ("C", 2), ("D",1)],          # 字典，和B相连的是A(5)，C(2),D(1)
    "C": [("A",1), ("B", 2), ("D",4), ("E",8)], # 字典，和C相连的是A(1)，B(2),D(4),E(8)
    "D": [("B",1), ("C", 4), ("E",3), ("F",6)], # 字典，和D相连的是B(1)，C(4),E(3),F(8)
    "E": [("C",8), ("D", 3)],                   # 字典，和E相连的是C(8)，D(3)
    "F": [("D",6)]                              # 字典，和F相连的是D(6)
}

Dijkstra("A", graph)

defaultdict(<function __main__.Dijkstra.<locals>.<lambda>()>,
            {'A': 0, 'B': 3, 'C': 1, 'D': 4, 'E': 7, 'F': 10})

# 字符串匹配算法（KMP和BF）
- 解决的问题：给定两个串S=“s1s2s3 …sn”和T=“t1t2t3 …tn”，在主串S中寻找子串T的过程叫做模式匹配，T称为模式
- 朴素模式匹配BF：现在假设现在主串S匹配到i位置，模式串T匹配到j位置
    - 如果当前字符匹配成功（即S\[i\] = T\[j\]），则i++，j++，继续匹配下一个字符；
    - 如果失配（即S\[i\] != T\[j\]），令i=i-j+1，j = 0。相当于每次匹配失败时，i回溯到本次失配起始字符的下一个字符，j回溯到0。
- KMP算法的改进：每当从某个起始位置开始一趟比较后，在匹配过程中出现失配，不回溯i，而是利用已经得到的部分匹配结果，将一种假想的位置定位“指针”在模式上向右滑动尽可能远的一段距离到某个位置后，继续按规则进行下一次的比较。
- KMP算法流程：规定i是主串S的下标，j是模式T的下标。现在假设现在主串S匹配到i位置，模式串T匹配到j位置。
    - 如果j = -1，则i++，j++，继续匹配下一个字符；
    - 如果S\[i\] = T\[j\]，则i++，j++，继续匹配下一个字符
    - 如果j != -1，且S\[i\] != T\[j\]，则i不变，j = next\[j\]。此举等价为失配时，接下来模式串T要相对于主串S向右移动j-next\[j\]位
    - 注：next数组指除当前字符外（注意不包括当前字符）的“公共 前后缀 最长长度”（前缀中和后缀中相同的最长的那一个子串的长度）
    - 前缀：不包含最后末尾字符，且首字母为第1个字符的子串；后缀：不包含第一个字符，且尾字符为最后一个字符的子串
    - next数组的生成可以参看对应链接
    
    

## 暴力匹配(BF)

In [1]:
def BF(S, T):
    '''
    返回匹配上的S的起始位置
    '''
    i = 0
    j = 0
    maxS = len(S)
    maxT = len(T)
    while (i < maxS and j < maxT):
        if S[i] == T[j]:
            i += 1
            j += 1
        else:
            i = i-j+1 # 回到原来位置的下一个重新开始比较
            j = 0

    if j == maxT:
        return (i-j)
    else:
        return "匹配不上"
s = "abdcajsne"
t = "ajsne"
BF(s, t)

4

## KMP算法
kmp重点在于如何得到next数组
- https://blog.csdn.net/yyzsir/article/details/89462339

In [ ]:
def buildNext(S):
    Next = [None for _ in S]
    j = 0
    k = -1 # 表示“公共前后缀”的最长长度
    Next[j] = k # 初始化Next[0]=-1
    while j < len(S)-1:
        if k==-1 or S[j] == S[k]:   # 当k迭代到-1时都没有找到最短则应该让Next[j]=0，或者当S[k] == S[j]时，Next[j+1] = k+1
            j += 1
            k += 1
            Next[j] = k    # 记录下当前字符外的“公共前后缀最长长度”

            # ## 更好理解个更新方式是
            # Next[j+1] = k+1
            # j += 1
            # k += 1
        else:
            k = Next[k]    # 如果不相等，就找在前缀里面找
    return Next

def KMP(S, T):
    N = buildNext(S) # next列表
    i = 0
    j = 0
    maxS = len(S)
    maxT = len(T)
    while (i < maxS and j < maxT):
        if S[i] == T[j] or j == -1:
            i += 1  # S继续识别下一个字母是否匹配
            j += 1  
        else:
            j = N[j]    # j不用回到最开始0的位置，只用回到next[j]的位置，往后识别
    if j == maxT:
        return (i-j)
    else:
        return "匹配不上"
# s = "abaabca"
# t = "abca"
# KMP(s,t)

### 问题1
给定两个头节点的二叉树，返回其中一个是否为另一个的子结构(并不是查询是否为子树，而是值是否是另一个树的值得一部分)

- 将两个树进行先序序列化，查看两个序列化之后的列表是否具有包含关系采用KMP算法即可。
- 原因是：先序序列化的树结构是唯一的，只要有包含关系则一定能重构出相同的树结构。


In [ ]:
# 构造测试示例
class Node():
    def __init__(self,item):
        self.item=item
        self.left=None
        self.right=None
class Tree():
    def __init__(self):
        self._root = None 
    def addNode(self,item):
        node = Node(item)
        if self._root == None:
            self._root = node
            return

        cur = self._root
        queue = [cur]
        while queue:
            n = queue.pop(0)
            if n.left != None:
                queue.append(n.left)
            else:
                n.left = node
                break
            if n.right != None:
                queue.append(n.right)
            else:
                n.right=node
                break
tree1 = Tree()
tree1.addNode("可以")
tree1.addNode(1)
tree1.addNode(3)
tree1.addNode("12")
tree1.addNode("wei")
tree1.addNode("静静儿")

tree2 = Tree()
tree2.addNode(3)
tree2.addNode("静静儿")

In [ ]:
def savetree(root, a):
    if root == None :
        a.append(None)
    else:
        a.append(root.item)
        savetree(root.left, a)
        savetree(root.right, a)
    return "储存完毕"
a1 = []
a2 = []
savetree(tree1._root, a1)
savetree(tree2._root, a2)
print(a1)
print(a2)
KMP(a1, a2)

# BFPRT算法
在无序数组中求第k小的数(没有第0小的数)。

- 方法一：
    - 利用荷兰国旗的算法思想，将数组分成3个部分，小于标定值，等于标定值，大于标定值的，我们可以得到在整个数组中，等于标定值的数在数组中排在第几名(因为左边全是小于标定值的数)。标定值是随机选的。
    - 如果我们的k不在等于标定值的范围内，则意味着第k大的数要么在左边区域，要么在右边区域，继续对不同区域做荷兰国旗问题就可以了。

- 方法二：bfprt算法
    - 后续仍然是荷兰国旗思想，但是选取标定值是通过挑选来得到的，而不是随机得到
    - 如何筛选这个标定值呢？
        - 将整个arr数组每五个位置分为一组，最一组的元素个数可以小于5
        - 在每个分组中由小到大排序(只用排组内)，取出每组排序后的中位数拿出来，构成长度大约为N/5的数组m。
        - 最后取出m的数组中的中位数(也就是第N/10小的数)，作为我的标定值来进行接下来的荷兰国旗算法。


In [ ]:
import random

def partition(arr, value):
    less = -1         # 小于value的最后一个位置     
    more = len(arr)   # 大于value的第一个位置
    L = 0             # 用于锁定等于value的值
    while L < more:   
        if arr[L] < value:
            less += 1
            arr[L], arr[less] = arr[less], arr[L]
            L += 1
        elif arr[L] > value:
            more -= 1
            arr[L], arr[more] = arr[more], arr[L]
        else:               
            L += 1
    return (less+1, more-1)

# 在arr中[L,R]区域中找到arr中的第k小的数(k=0,1,2,3,4...)
# 但是得保证[L,R]中一定有第k小的数
def getLessNumber(arr, k):
    if len(arr) == 1:
        return arr[0]
    value = arr[random.randint(0,len(arr))]
    (less, more) = partition(arr, value)    #荷兰国旗问题会改变原数组
    if less <= k and k <= more:
        return arr[k]
    elif k < less:
        return getLessNumber(arr[:less], k)
    elif k > more:
        return getLessNumber(arr[more+1:], k-more-1)

# 测试示例
arr = [1,333,12,23,41,2,3,4,4,5,12,234,3,3,24,567,5,2,1]
arrcopy = arr.copy()
getLessNumber(arrcopy, 4)#返回第5小的数

In [ ]:
def bfprt(arr, L, R, k):
    if L == R:
        return arr[L]
    value = getValue(arr[L:(R+1)])          #在指定范围内找到最优的标定值
    (less, more) = partition(arr, value)    #荷兰国旗问题会改变原数组
    if less <= k and k <= more:
        return arr[k]
    elif k < less:
        return bfprt(arr, L, less-1, k)
    elif k > more:
        return bfprt(arr, more+1, R, k)

def partition(arr, value):
    less = -1         # 小于value的最后一个位置     
    more = len(arr)   # 大于value的第一个位置
    L = 0             # 用于锁定等于value的值
    while L < more:   
        if arr[L] < value:
            less += 1
            arr[L], arr[less] = arr[less], arr[L]
            L += 1
        elif arr[L] > value:
            more -= 1
            arr[L], arr[more] = arr[more], arr[L]
        else:      
            L += 1
    return (less+1, more-1)


# 用来得到荷兰国旗算法需要的最佳值
def getValue(arr):
    m =[]
    mNumber = len(arr)//5 if len(arr)%5 == 0 else len(arr)//5+1
    for i in range(mNumber):
        L = i*5
        R = min((i+1)*5, len(arr))
        m.append(getMiddle(arr[L:R]))
    return bfprt(m, 0, len(m)-1, len(m)//2)

# 计算中位数
def getMiddle(arr):
    arrcopy = arr.copy()
    arrcopy.sort()
    return arrcopy[len(arr)//2]

# 测试示例
arr = [1,333,12,23,41,2,3,4,4,5,12,234,3,3,24,567,5,2,1]
arrcopy = arr.copy()
bfprt(arrcopy, 0, len(arrcopy)-1, 4)#返回第5小的数


# 并查集
我们可以用并查集来判断一个森林中有几棵树、某个节点是否属于某棵树等。并查集主要有两个功能，第一是查询x属于哪个集合，第二个是合并x和y使得他们俩在一个集合里。

分析:
- 每个“集合”(连通图)使用一个**代表元**来代表这个集合，就好比树的根节点root代表整颗树。主元不一样意味着x，y属于不同的“集合”。
- 使用一个数组pre表示上级列表，例如pre[i] = j，表示i的上级是j。如果pre[i] = i，意味着i是作为这个集合的**代表元**。根据初始化pre的不同，确定找到主元的条件也是不同的，例如：初始化pre的时候，每个位置定义是pre[k] = 0 ，因为开始的时候每个各元素之间都没有关系因此，每个元素都是自己的**代表元**，此时如果有pre[h] = 0 ，才说明主元为h。
- find（x）：找到x的主元，那就只能一级一级的往上找，先查看x的上级pre[x]（假设为j），再查看pre[j]（假设j的上级为i），再接着查看pre[i]（结果发现i
的上级是i，说明找到了主元了），因此x的主元就是i。
- join（x, y）：当i，j产生关系了，此时要考虑i和j是不是在同一个集合里（也就是i和j的代表元是否相同）。x的代表元fx，y的代表元fy都是可以通过find函数找到的
    - 若fx = fy，那么不用做任何操作。（因为x，y已经是一个团体里的了）
    - 若fx != fy，则可以定义fx是fy的上级（或者fy是fx的上级），从而更新pre[fy] = fx（或pre[fx] = fy），要注意我们更新的是x，y的代表元关系而不是x，y的关系


应用题型：
- 判断图的连接是否为环，求连通分支个数（每个子图中相互可以直接或间接到达），集合个数

In [ ]:
pre = [i for i in range(10)] # 定义了一个10人的并查集


## 查找函数
def find(x):
    while pre[x] != x: # 若pre[x]不等于x说明x不是主元，向上查找
        x = pre[x]
    return x


## 并函数
def union(x, y):
    fx = find(x)
    fy = find(y)
    if fx != fy:
        pre[fx] = fy # 定义fx的上级为fy

union(4, 2)
union(3, 2)
union(3, 5)


## 优化find()
我们在使用join(x,y)进行合并是随机指定上级（上面的例子是指定后者为上级），但是这种方式会使得最终得到的结构很不稳定，例如
```python
join(4, 2)
join(3, 2)
join(3, 5)
```
执行之后，我们查找3的代表元需要向上找2次，而使用
```python
join(4, 2)
join(3, 2)
join(5, 3)
```
我们查找3的代表元只需要向上找1次。因此如果能对查找路径进行优化将有更好效率。于时我们可以使用递归函数，在查找路径的同时，更新当前位置的上级, 直接更新当前节点代表元为父节点, 而不是连接在某个代表元子节点后面（层数会少很多）


In [ ]:
### 压缩路径的方法
def find(x):
    if pre[x] == x:
        return x
    else:
        pre[x] = find(pre[x])
        return pre[x]


## 并查集模板(一)
下面我们给出这种数据结构的最终模板, 其实我们还可以收集每个代表元下子节点的个数

In [ ]:
class UnionFind:
    def __init__(self, n) -> None:  # 使用并查集需要知道有多少个节点
        self.pre = list(range(n))
        self.size = [1] * n         # size[i]表示节点i下的子节点数

    # 使用路径压缩find
    def find(self, x):
        fx = self.pre[x]
        if fx == x:
            return x
        else:
            if fx != self.pre[fx]:  # 如果fx不是代表元, 则需要将fx的子节点个数-1
                self.size[fx] -= 1
            self.pre[x] = self.find(fx)
            return self.pre[x]
    
    # 将节点x, 与节点y合并(返回是否完成合并, 如果为False表明未合并)
    def union(self, x, y) -> bool:
        fx = self.find(x)
        fy = self.find(y)
        if fx == fy:    # 说明为x, y在同一个代表元下
            return False
        elif self.size[fx] >= self.size[fy]:    # 合并后将少节点的集合合并到多节点中
            self.pre[fy] = fx
            self.size[fx] += self.size[fy]
        elif self.size[fx] < self.size[fy]:
            self.pre[fx] = fy
            self.size[fy] += self.size[fx]
        return True


## 并查集模板(二)
在一些算法题中, 我们可能只需要并查集合并节点, 并得到连通块数量, 因此我们完全可以更简单是的只使用find函数, 合并动作由我们自己进行

In [ ]:
nums = [3,12,31]
n = len(nums)
fa = list(range(n))  # 每个下标都是自己的父节点

# 查找x的父节点(同时压缩路径)
def find(x):
    if fa[x] != x:
        fa[x] = find(fa[x])  # 压缩路径, x指向指向根节点
    return fa[x]

x = 1
y = 2
# 找父节点后, 查看两个节点的父节点是否为同一个, 然后进行合并
fx = find(x)
fy = find(y)
if fx != fy:
    fa[fx] = fy
    # 这里执行合并后的一些统计操作, 比如连通块-1, 根节点下个数增加等等


## [每日一题 # 45](20每日一题.ipynb)

# Floyd算法
Floyd算法是用于求解有向图(或无向图)中，任意两点间的最短距离。听起来和Dijkstra算法很像，但后者是单源的，即每次得到的是从一个点到其他所有点的最短路径，而Floyd算法则是会得到所有的两点间的最短路径。但从复杂度上看Floyd算法是$O(n^3)$, 而Dijkstra算法是$O(n^2)$(这也是很自然的, 因为毕竟Dijkstra只计算了一个点)。https://blog.csdn.net/qq_34989804/article/details/82149495

分析：
- Floyd算法的思想是：如果A-B的直接距离不是最短的，那意味着我们可以通过第三个点中转使得A-B的距离减少，比如一个点C可以做为中转点使得A-C-B的距离小于A-B。但需要注意的是我们不一定只中转一次，也有可能是A-B通过两个中转点C,D实现, A-C-D-B的距离最短。
- 在初始的邻接矩阵中, 我们假设只通过A点作为中转点, 遍历所有节点，观察是否是有edge[i][A] + edge[A][j] < edge[i][j], 如果成立，那么意味着i-j的最短距离可以用A点作为中转点从而减少，此时将邻接图 edge[i][j]位置更改为最短距离。
- 完成上一步更新后的邻接矩阵，即为**可以**通过A点作为中介点任意两点间的最短路径。接下来在上一步邻接矩阵的基础上继续加入点B作为中介点，按照上述规则完成更新。
- 在上一步更新后的基础上再加入C点,D点...知道所有的点都加入后，得到的邻接矩阵就是：允许通过所有点作为中介点，任意两点之间的最短距离。

In [17]:
import copy
# 通过字典来得到权有向图
graph = {
    "A":{"B":5, "C":1},              # 字典，和A相连的是B(5)，C(1)
    "B":{"A":5, "C":2, "D":1},       # 字典，和B相连的是A(5)，C(2),D(1)
    "C":{"A":1, "B":2, "D":4, "E":8},# 字典，和C相连的是A(2)，B(2),D(4),E(8)
    "D":{"B":1, "C":4, "E":3, "F":6},# 字典，和D相连的是B(1)，C(4),E(3),F(8)
    "E":{"C":8, "D":3},              # 字典，和E相连的是C(8)，D(3)
    "F":{"D":6}                      # 字典，和F相连的是D(6)
}

edges = [[graph[i].get(j,float("inf")) if i != j else 0 for j in graph] for i in graph]  ## 变成n*n邻接矩阵的形式

def floyd(edges):
    edges_copy = copy.deepcopy(edges)
    length = len(edges)
    for k in range(length):  ## 每次将一个点作为中介点更新图
        ## 以下是在更新图
        for i in range(length):
            for j in range(length):
                edges_copy[i][j] = min(edges_copy[i][j], edges_copy[i][k]+edges_copy[k][j])
    return edges_copy


floyd(edges)

[[0, 3, 1, 4, 7, 10],
 [3, 0, 2, 1, 4, 7],
 [1, 2, 0, 3, 6, 9],
 [4, 1, 3, 0, 3, 6],
 [7, 4, 6, 3, 0, 9],
 [10, 7, 9, 6, 9, 0]]

## [每日一题 # 96](20每日一题.ipynb)

# Kruskal算法 (最小生成树)
Kruskal 算法是一种贪心算法，用于求解无向图的最小生成树。(最小生成树定义: 最小生成树是一棵包含图中所有顶点的树，并且树上边的权重之和最小).其算法核心思路: 使用贪心策略, 每次选择一条权值最小的边，并且这条边不能使已经选择的边构成回路，直到生成树包含所有顶点。
- 将边(node1, node2, weight)按照weight进行排序, 然后从最小权重开始构建最小生成树。
- 判断是否构成回路可以使用并查集, 判断两个节点是否在同一个集合中即可

下面是使用简单并查集实现的Kruskal算法模板

In [ ]:
def kruskal(edges, n):
    '''
    params:
        edges: [(weight, u, v)...] 每个元素表示边权值，u，v
        n: 表示节点个数
    return:
        tuple: 包含两个元素
        - list of tuple: 最小生成树的边，每个边表示为 (u, v, weight) 的元组
        - int: 最小生成树的总权值
    '''
    edges.sort()

    # 构造简单并查集
    fa = list(range(n))  # 每个下标都是自己的父节点
    # 查找x的父节点(同时压缩路径)
    def find(x):
        if fa[x] != x:
            fa[x] = find(fa[x])  # 压缩路径, x指向指向根节点
        return fa[x]
        

    mst = []  # 记录最小生成树
    total_weight = 0

    for weight, x, y in edges:
        fx, fy = find(x), find(y)
        if fx != fy:
            fa[fx] = fy   # 合并
            mst.append((x, y, weight))
            total_weight += weight
    return mst, total_weight